# Detik

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import random

def filter_ads(soup, ad_classes):
    ad_divs = soup.find_all(class_=ad_classes) 
    for ad_div in ad_divs:
        ad_div.extract()

train_csv_file = open('training_data4.csv', 'w', newline='', encoding='utf-8')
test_csv_file = open('True.csv', 'w', newline='', encoding='utf-8')

train_csv_writer = csv.writer(train_csv_file)
test_csv_writer = csv.writer(test_csv_file)

train_csv_writer.writerow(['Title', 'URL', 'Text', 'Label'])
test_csv_writer.writerow(['Title', 'URL', 'Text'])

# List to store scraped data
data = []

# List of target dates
target_dates = ["03/20/2024", "02/14/2024",
                "02/04/2024", "01/21/2024", 
                "01/07/2024", "12/22/2023", 
                "12/12/2023", "10/25/2023",
                "10/19/2023"]

# DETIK
for target_date in target_dates:
    page = 1
    while page <= 2: 
    # while True:
        url = f"https://news.detik.com/pemilu/indeks/{page}?date={target_date}"
        print(target_date, page)
        page_content = requests.get(url)
        soup = BeautifulSoup(page_content.text, 'html.parser')

        # Title
        titles = soup.find_all('h3', class_='media__title')

        filter_ads(soup, 'media__image')

        # Link
        links = soup.find_all('a', class_='media__link')

        # Check if there are no more news articles for the current date
        if not titles:
            break

        for title, link in zip(titles, links):
            title_text = title.text.strip()
            url = link.get('href')

            ''' Content '''
            '''================================================================'''
            # Link content
            news_page = requests.get(url)
            news_soup = BeautifulSoup(news_page.content, 'html.parser')

            # Filter out advertisement divs
            filter_ads(news_soup, 'parallaxindetail scrollpage')
            filter_ads(news_soup, 'staticdetail_container')
            filter_ads(news_soup, 'lihatjg')
            filter_ads(news_soup, 'detail__body-tag mgt-16')

            text_content = news_soup.find('div', class_='detail__body-text itp_bodycontent')
            '''================================================================'''

            if text_content:
                text = text_content.text.strip()
            else:
                text = "No content found for: " + url

            # Assign 80% of the data to the training set and 20% to the testing set
            if random.random() < 0.8:
                data.append((title_text, url, text, 1))
            else:
                test_csv_writer.writerow([title_text, url, text])
        
        # check progress
        page += 1

random.shuffle(data)

# train
for item in data:
    train_csv_writer.writerow(item)

# Close CSV files
train_csv_file.close()
test_csv_file.close()


clean dataset

In [6]:
import csv

def filter_title(title):
    filtered_title = title
    filtered_title = filtered_title.replace("'", '')
    filtered_title = filtered_title.replace('"', '')
    return filtered_title.strip()

def filter_text(text):

    filtered_text = text
    filtered_text = ' '.join(filtered_text.split())

    unwanted_texts = ['Simak berita selengkapnya di halaman selanjutnya. Halaman 1 2 Selanjutnya', ': Halaman 1 2 3 Selanjutnya']

    for unwanted_text in unwanted_texts:
        filtered_text = filtered_text.replace(unwanted_text, '')
    filtered_text = filtered_text.replace("'", '')
    filtered_text = filtered_text.replace('"', '')
    return filtered_text.strip()

def filter_csv(input_file, output_file):
    with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            for row in reader:
                row[0] = filter_title(row[0])
                row[2] = filter_text(row[2])
                writer.writerow(row)

input_file = 'training_data4.csv'
output_file = 'Cleaned/training_data4.csv'

input_file2 ="True.csv"
output_file2 = 'Cleaned/True.csv'

filter_csv(input_file, output_file)
filter_csv(input_file2, output_file2)